# Part 4 - Generator / Summarizer
In this notebook, we implement the 3rd transformer, in charge or generating a review of the best product per category, from a batch of reviews and sentiments

## Imports

In [2]:
import os
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoModelForCausalLM
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset
import torch
from datetime import datetime
from tqdm import tqdm
tqdm.pandas()  # Initialize tqdm with pandas
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# import helpers module
import helpers
import importlib
importlib.reload(helpers)

SEP = 100 * '-'

2024-10-15 17:35:19.143212: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-15 17:35:19.143212: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-15 17:35:19.240282: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-15 17:35:19.452168: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-15 17:35:21.993406: W tensorflow/compiler/tf2

In [3]:
# load HugginFace token to environment
if not os.environ.get('HF_TOKEN'):
    os.environ['HF_TOKEN'] = input('Enter API token for Hugging Face: ')
else:
    print('Hugging Face token already loaded to environment')

## Load dataset

In [4]:
# load dataset
data = helpers.load_pickled_dataset('pickle/data_sentiment_analysis.pkl')

helpers.print_random_product_sheet(data)

Dataset loaded from pickle/data_sentiment_analysis.pkl.
----------------------------------------------------------------------------------------------------
[name] Amazon - Amazon Tap Portable Bluetooth and Wi-Fi Speaker - Black,,,
Amazon - Amazon Tap Portable Bluetooth and Wi-Fi Speaker - Black,,,
----------------------------------------------------------------------------------------------------
[brand] Amazon
----------------------------------------------------------------------------------------------------
[categories] Stereos,Remote Controls,Amazon Echo,Audio Docks & Mini Speakers,Amazon Echo Accessories,Kitchen & Dining Features,Speaker Systems,Electronics,TVs Entertainment,Clearance,Smart Hubs & Wireless Routers,Featured Brands,Wireless Speakers,Smart Home & Connected Living,Home Security,Kindle Store,Home Automation,Home, Garage & Office,Home,Voice-Enabled Smart Assistants,Virtual Assistant Speakers,Portable Audio & Headphones,Electronics Features,Amazon Device Accessories,iPo

## METHOD 1: Prompt Fine-Tuning
Let's see what we can get using only prompt fine-tuning. For this I will try [Mistral-7B](https://huggingface.co/mistralai/Mistral-7B-v0.3) in 4bit quantization for quick inference.

Using 4-bit quantization will help reduce memory usage significantly while retaining reasonable model performance.

In [5]:
!transformers-cli cache clear

2024-10-15 17:38:51.651129: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-15 17:38:51.651220: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-15 17:38:51.653330: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-15 17:38:51.664341: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-15 17:38:53.179161: W tensorflow/compiler/tf2

### Load Mistral-7b 4bits

In [15]:
from transformers import LlamaTokenizer

model_name = "mistralai/Mistral-7B-v0.3"
tokenizer = LlamaTokenizer.from_pretrained(model_name)

# load 4bit quantization model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)

# test model
inputs = tokenizer("Hello my name is", return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Hello my name is Katie and I am a 20 year old student at the University of North Carolina at Chap


### Prompt Template
Let's work on a prompt template so the model extract information from reviews and summarize them

In [19]:
prompt = """Summarize the following product review in one sentence, highlighting the main points.
Then, provide a list of pros and cons and a short overall conclusion.

### Review: 
{review_text}

### One-sentence summary, pros and cons and conclusion:

"""



In [20]:
def generate_summary(prompt, review):
    inputs = tokenizer(prompt.format(review_text=review), return_tensors="pt")
    
    outputs = model.generate(
        inputs.input_ids,
        max_new_tokens=100,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id
    ).to(device)
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

review_text = "This laptop has a fantastic display and long battery life, but the keyboard is a bit cramped, and it tends to get hot during heavy use. Overall, it performs well for everyday tasks and is good value for the price."

result = generate_summary(prompt, review_text)

print(result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Summarize the following product review in one sentence, highlighting the main points.
Then, provide a list of pros and cons and a short overall conclusion.

### Review: 
This laptop has a fantastic display and long battery life, but the keyboard is a bit cramped, and it tends to get hot during heavy use. Overall, it performs well for everyday tasks and is good value for the price.

### One-sentence summary, pros and cons and conclusion:

- Pros:
  - Fantastic display
  - Long battery life
  - Good value for the price
- Cons:
  - Keyboard is a bit cramped
  - Tends to get hot during heavy use
- Conclusion:
  - Performs well for everyday tasks




































